In [1]:
import pandas as pd
import sqlite3 as sl

In [6]:
# открываем файл с базой данных
sql = sl.connect('Boston_Celtics_Team.db')

In [7]:
# таблица игроков команды - 'players'
full_name = ['Jayson Tatum', 'Jaylen Brown', 'Al Horford', 'Malcolm Brogdon', 'Marcus Smart',
             'Derrick White', 'Luke Kornet', 'Payton Pritchard','Grant Williams', 'Sam Hauser']

dob = ['10/24/1996', '3/3/1998', '6/3/1986', '12/11/1992', '3/6/1994',
       '7/2/1994', '10/17/1997', '8/8/1988', '11/30/1998', '7/1/1991']

height = [198, 207, 210, 195, 195, 195, 210, 186, 184, 185]
weight = [101, 95, 109, 98, 90, 88, 93, 89,92, 90]
role = ['SG', 'SF', 'C', 'PG', 'PG', 'C', 'PF', 'PF', 'C', 'PG']

player_df = pd.DataFrame({'name': full_name, 'DOB': dob, 'Role': role, 'Height': height, 'Weight': weight })

In [8]:
player_df.to_sql('players', sql, if_exists='replace')

10

In [101]:
#df = pd.read_sql("SELECT * from players", sql) #можем проверить, что получилось

In [9]:
# таблица амплуа игроков - 'Role_Player', один ко многим с 'Player'
id_role = ['PG', 'SF', 'C', 'SG', 'PF']
role = ['Point_Guard', 'Small_Forward', 'Center', 'Shooting_Guard', 'Power_Forward']

role_df = pd.DataFrame({'Id_Role': id_role, 'Role': role})
role_df.set_index('Id_Role')
role_df.to_sql('role_player', sql, if_exists='replace')

5

In [ ]:
# df = pd.read_sql("SELECT * from Role_Player", sql, index_col='Id_Role') #можем проверить, что получилось

In [10]:
# список комманд оппонентов
opponent_teams = ['Atlanta Hawks', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
                  'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets',
                  'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers','Memphis Grizzlies', 'Miami Heat',
                  'Milwaukee Bucks', 'Minnesota Timberwolves','New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder',
                  'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings',
                  'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
nba_teams = pd.DataFrame({'Name_Team': opponent_teams})
nba_teams.to_sql('NBA_Teams', sql, index_label="id", if_exists='replace')

29

In [60]:
# df = pd.read_sql("SELECT * from NBA_Teams", sql) #можем проверить, что получилось

Дальше, есть небольшой csv файл со статистикой Соперник/Дата/Игрок_Бостон/Кол_минут/Кол_очков, который нужно добавить в БД.

Связь 'Players' и 'Matches' будет 'many-to-many'.

In [11]:
# 'matches'
df = pd.read_csv('Games.csv', encoding='utf-8',  delimiter=';')
df.to_sql('matches', sql, index_label="id", if_exists='replace')

60

In [85]:
# df = pd.read_sql("SELECT * from Matches", sql, index_col='id') #можем проверить, что получилось

In [12]:
pd.read_sql('SELECT * FROM players', sql)

,index,name,DOB,Role,Height,Weight
0,0,Jayson Tatum,10/24/1996,SG,198,101
1,1,Jaylen Brown,3/3/1998,SF,207,95
2,2,Al Horford,6/3/1986,C,210,109
3,3,Malcolm Brogdon,12/11/1992,PG,195,98
4,4,Marcus Smart,3/6/1994,PG,195,90
5,5,Derrick White,7/2/1994,C,195,88
6,6,Luke Kornet,10/17/1997,PF,210,93
7,7,Payton Pritchard,8/8/1988,PF,186,89
8,8,Grant Williams,11/30/1998,C,184,92
9,9,Sam Hauser,7/1/1991,PG,185,90


In [13]:
pd.read_sql('SELECT * FROM Matches', sql).head()

,id,Team,Date,Boston_Team,Mins,PTS
0,0,Philadelphia 76ers,22.10.2018,Jayson Tatum,38,35
1,1,Philadelphia 76ers,22.10.2018,Jaylen Brown,38,35
2,2,Philadelphia 76ers,22.10.2018,Marcus Smart,36,14
3,3,Philadelphia 76ers,22.10.2018,Derrick White,24,2
4,4,Philadelphia 76ers,22.10.2018,Al Horford,23,6


In [14]:
# Просто посмотрим, как играл лидер команды )
pd.read_sql(
    '''SELECT Boston_Team, Mins, PTS, Team
         FROM Matches
        WHERE Boston_Team='Jayson Tatum'
     ORDER BY PTS DESC
    ''', sql)

,Boston_Team,Mins,PTS,Team
0,Jayson Tatum,39,39,Memphis Grizzlies
1,Jayson Tatum,38,35,Philadelphia 76ers
2,Jayson Tatum,37,34,Denver Nuggets
3,Jayson Tatum,36,29,Miami Heat
4,Jayson Tatum,35,26,Chicago Bulls
5,Jayson Tatum,33,19,Atlanta Hawks


In [15]:
# попролбуем что-то посложнее )
# получим небольшую сводную таблицу с ФИО_игрока, его амплуа, вычислим: сколько всего мин играл, сколько очков набрал и
# среднее количество очков за матч. И посмотрим только тех, кто играл больше 25 минут за матч в среднем.
pd.read_sql(
    '''SELECT Boston_Team,
              players.Role AS Role,
              SUM(Mins)    AS Total_Mins,
              SUM(PTS)     AS Total_Points,
              (SUM(PTS)/COUNT(DISTINCT Team)) AS Average_PTS
         FROM Matches  
         JOIN players ON players.Name=Matches.Boston_Team
     GROUP BY Boston_Team
       HAVING (SUM(Mins)/COUNT(DISTINCT Team)) > 25
    ''', sql)


,Boston_Team,Role,Total_Mins,Total_Points,Average_PTS
0,Al Horford,C,174,66,11
1,Grant Williams,C,162,58,9
2,Jaylen Brown,SF,203,152,25
3,Jayson Tatum,SG,218,182,30
4,Marcus Smart,PG,159,52,10


#### Сил на графики не хватило :(